In [3]:
import pandas as pd

In [3]:
import pandas as pd
import re
import os

def process_cct_data(file_path):
    """
    Process CCT data from the specified Excel file and create a combined dataset.
    
    Args:
        file_path (str): Path to input Excel file
    
    Returns:
        DataFrame: Combined DataFrame with all sentences
    """
    all_sentences = []
    
    print(f"Processing file: {file_path}")
    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    
    # Process each sheet
    for sheet in xls.sheet_names:
        print(f"  Processing sheet: {sheet}")
        df = pd.read_excel(xls, sheet_name=sheet)
        
        # Strip spaces from column names
        df.columns = df.columns.str.strip()

        # Find the essay column dynamically (assuming it contains "Essay" in the name)
        essay_column = [col for col in df.columns if "Essay" in col]
        if not essay_column:
            print(f"  No essay column found in sheet {sheet}. Skipping.")
            continue
        essay_column = essay_column[0]

        # Identify CCT columns dynamically (those ending with "FINAL")
        cct_columns = [col for col in df.columns if col.endswith("FINAL")]
        print(f"  Found {len(cct_columns)} CCT columns")

        # Process each row (student essay)
        for idx, row in df.iterrows():
            essay_id = row.get("Alma ID", f"Unknown-{idx}")
            essay_text = row[essay_column]
            
            # Skip if essay is not a string
            if not isinstance(essay_text, str):
                continue
                
            # Clean the essay text and split into sentences
            cleaned_essay = essay_text.replace("†", "").strip()
            all_essay_sentences = [sent.strip() for sent in re.split(r'[.!?]', cleaned_essay) if sent.strip()]
            
            # Keep track of which sentences are assigned to CCT categories
            sentence_to_ccts = {}
            
            # Process CCT columns
            for cct_col in cct_columns:
                cct_text = row[cct_col]
                cct_name = cct_col.replace(" FINAL", "").strip()
                
                # Skip if not a string (e.g., 0 or NaN)
                if not isinstance(cct_text, str):
                    continue
                
                # Extract segments between /%/ markers
                segments = re.findall(r'/%/(.*?)/%/', cct_text)
                
                for segment in segments:
                    segment = segment.strip()
                    if segment and len(segment.split()) >= 5:  # Minimum sentence length
                        if segment in sentence_to_ccts:
                            sentence_to_ccts[segment].append(cct_name)
                        else:
                            sentence_to_ccts[segment] = [cct_name]
            
            # Add sentences with CCT labels to the results
            for sentence, ccts in sentence_to_ccts.items():
                all_sentences.append({
                    "sentence": sentence,
                    "CCTs": ", ".join(sorted(set(ccts))),
                    "essay_id": essay_id
                })
            
            # Find sentences not assigned to any CCT category and add to Class 0
            for sentence in all_essay_sentences:
                if sentence not in sentence_to_ccts and len(sentence.split()) >= 5:  # Minimum length
                    all_sentences.append({
                        "sentence": sentence,
                        "CCTs": "Class 0",
                        "essay_id": essay_id
                    })

    # Convert to DataFrame
    combined_df = pd.DataFrame(all_sentences)
    print(f"Total extracted sentences: {len(combined_df)}")
    
    # Count by category
    cct_counts = {}
    for _, row in combined_df.iterrows():
        ccts = row["CCTs"].split(", ")
        for cct in ccts:
            cct_counts[cct] = cct_counts.get(cct, 0) + 1
    
    # Print category statistics
    print("\nSentence counts by category:")
    for cct, count in sorted(cct_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {cct}: {count}")
    
    return combined_df

# Example usage in a notebook:
file_path = "Alma-PHYS122-F19-post.xlsx"
combined_df = process_cct_data(file_path)
combined_df.to_csv("processed-annotated-data.csv", index=False)

Processing file: Alma-PHYS122-F19-post.xlsx
  Processing sheet: Fall 2019 - PHYS 112 Lab
  Found 11 CCT columns
Total extracted sentences: 294

Sentence counts by category:
  Class 0: 228
  Navigational: 28
  Aspirational: 26
  Attainment: 11
  Perserverance: 5
  Spiritual: 4
  Social: 2
  Familial: 2
  Filial Piety: 1
